# 要約 
このノートブックは、Kaggleにおける「20の質問」ゲーム用にGemmaモデルを使用するためのガイドです。主な目的は、新しいユーザーがGemmaを設定し、使用する際の手順を示すことです。本ノートブックは、LLM（大規模言語モデル）の利用を容易にする助けとなり、特にGemmaの実装に焦点を当てています。

### 取り組んでいる問題
ノートブックは、Gemmaモデルのインストールから、モデルの読み込み、簡単な質問応答の実行に至るまでの全プロセスをステップバイステップで解説しています。特に、Gemmaを活用して「20の質問」形式のゲームにおけるインタラクションを実現することに特化しています。

### 使用している手法とライブラリ
1. **ライブラリのインストール**:
    - `immutabledict`と`sentencepiece`などのPythonパッケージを使用しています。これらは、Gemmaモデルの動作に必要な依存関係です。

2. **Gemmaのクローンと設定**:
    - Gitを使用して`gemma_pytorch`リポジトリをクローンし、関連ファイルを指定のディレクトリに移動しています。

3. **モデルの読み込み**:
    - PyTorchを使用し、GemmaモデルをGPUに読み込む設定がなされています。`GemmaForCausalLM`クラスを用いてモデルの設定と重量の読み込みを行っています。

4. **質問応答の実施**:
    - モデルの`generate`メソッドを使用して、プロンプトに対する応答を生成しています。この部分では、出力の多様性を制御するための`temperature`や`top_p`、`top_k`の設定が行われています。

5. **インストラクションチューニング**:
    - モデルに「20の質問」ゲームのAIアシスタントとしての行動を促すプロンプトを用いることで、具体的なタスクに特化した応答を得る技術を実装しています。

このノートブックは、Gemmaモデルの設置から質問応答の実行まで、一連のプロセスを分かりやすく解説しており、実際の使用例を通じて読者に対して実践的なガイドを提供しています。

---


# 用語概説 
以下は、提供されたJupyter Notebookの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。

1. **Gemma**:
   - 特定のLLM（大規模言語モデル）で、生成AIタスク用に設計されたモデル。このノートブックでは、Gemmaモデルを用いて「20の質問」ゲームのような対話形式のアプリケーションを構築するための手順が示されています。

2. **プロンプト (prompt)**:
   - モデルに与える入力文。モデルはこのプロンプトに基づいて出力を生成するため、プロンプトの設計は結果に大きな影響を与えます。

3. **トークナイザー**:
   - テキストをモデルが理解できる形式（トークン）に変換するツール。トークンは、単語や文の構成要素を小さく分割したもので、モデルはこれらのトークンを処理します。

4. **temperature**:
   - 生成するテキストの多様性を制御するパラメーター。低い値（例: 0.01）を設定すると、モデルはより規則的で確実な（ただし創造的ではない）出力を生成し、高い値では多様で創造的な出力を生成します。

5. **top_p (nucleus sampling)**:
   - 確率的サンプリング手法の一つで、累積確率がtop_pの値を超えるトークンのみを考慮して生成を行います。これにより、予測されるトークンの中からより有効な選択肢を絞ることができます。

6. **top_k**:
   - サンプリング手法で、モデルが次のトークンとして選ぶことができる可能性のある上位k個のトークンに基づいて出力を生成します。これによって出力の多様性が制御されます。

7. **CUDA**:
   - NVIDIAのGPUを利用するためのプラットフォーム。PyTorchでは、モデルをCUDA対応デバイスに移動させることで、計算を高速化できます。

8. **immutabledict**:
   - 変更不可能な辞書型データ構造。通常の辞書と異なり、内容を変更できないため、データの不変性が保証されます。

9. **instruction tuning**:
   - モデルに特定のタスクを実行させるための調整手法。プロンプトエンジニアリングと密接に関連しており、ユーザーの要求に基づいてモデルの出力を最適化します。

10. **evaluation mode (評価モード)**:
    - モデルが訓練されずにデータを処理する際の状態。評価モードでは、ドロップアウトやバッチ正規化などの特定の訓練時の操作が無効になります。

これらの用語は、初心者が特定のコンセプトを理解する際に役立つ基盤となる情報です。ノートブックの内容を理解するためや、Gemmaモデルを活用する際に重要な要素です。

---


このノートブックは、KaggleでGemmaを使用するのが初めての方のためのものです。  
その使い方をステップバイステップで見ていきましょう。

**謝辞**  
このノートブックは[LLM 20 Questions スターターノートブック](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)に基づいています。

始める前に、GPUの設定を確認してください。

<img width="400px" src=attachment:9319be98-0139-41f9-a741-6b606b7cadc6.png>

---

## Gemmaモデルの追加

1. `+ Add Input`ボタンをクリックします。
<img src=attachment:6dabe5ed-9e7e-437b-be73-ee0c3b3a2afe.png width="400">

2. `Gemma`モデルを検索します。
<img src=attachment:68d9d1fa-84f4-4c37-acf9-f5cb2b825fcb.png width="400">

3. パラメーターを設定します。（私は*2b-it*を使用しました。なぜなら*7b*はテストが遅すぎるからです）
<img src=attachment:c7f1744e-08df-4ba7-ab61-4b0cd4e4e6fd.png width="400">

4. モデルが追加されたことが確認できます。
<img src=attachment:056f294d-eee3-4bbb-b68b-6bc991a9c746.png width="400">

In [ ]:
%%bash
# 入力ファイルは`/kaggle/input`ディレクトリにあります。
# `tree`コマンドを使用して、指定したディレクトリ内のファイルとフォルダーの構造を表示します。
tree /kaggle/input/gemma/pytorch

# Gemmaをコーディングするための設定

Gemmaモデルを使用するためには、いくつかの設定が必要です。

## 依存関係のインストール

In [ ]:
%%bash
# `/kaggle/working/submission/lib`ディレクトリにPythonパッケージをインストールします。
mkdir -p /kaggle/working/submission/lib
# immutabledictとsentencepieceパッケージを指定したディレクトリにインストールします。
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# `gemma_pytorch`リポジトリをダウンロードして、`/kaggle/working/submission/lib`ディレクトリに移動します。
cd /kaggle/working
# gitを使用してgemma_pytorchリポジトリをクローンします。出力は非表示にします。
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorchからgemmaのファイルを指定したディレクトリに移動します。
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%bash
# 結果を確認します。
# `tree`コマンドを使用して、指定したディレクトリ内のファイルとフォルダーの構造を表示します。
tree /kaggle/working/submission/lib

In [ ]:
import sys

# `/kaggle/working/submission/lib`ディレクトリ内のライブラリを使用するために、モジュールのパスを追加します。
# これにより、指定したディレクトリからライブラリをインポートできるようになります。
sys.path.insert(0, "/kaggle/working/submission/lib")

## Gemmaモデルの読み込み

In [ ]:
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# グローバルパラメーターの設定
# 使用可能な場合はGPUを使用し、そうでなければCPUを使用します。
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
# 使用するGemmaモデルのバリアントを指定します。
VARIANT = "2b-it"
# VARIANT = "7b-it-quant"  # 他のバリアントをコメントアウトとして使用できます。
# モデルの重みが格納されているパスを指定します。
WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{VARIANT}/2" # 入力モデルパス

# 現在のデバイスとモデルバリアントを表示します。
print("current device is", DEVICE)
print("current Gemma model variant is", VARIANT)

In [ ]:
# モデル設定を行います
def get_model_config(variant):
    # バリアントが"2b"の場合、2bモデル用の設定を取得します。
    if "2b" in variant:
        return get_config_for_2b()
    # バリアントが"7b"の場合、7bモデル用の設定を取得します。
    if "7b" in variant:
        return get_config_for_7b()
    # サポートされていないバリアントの場合、エラーを発生させます。
    raise ValueError(f"Not supported variant - {variant}")

# モデルの設定を取得します
model_config = get_model_config(VARIANT)
# トークナイザーのモデルファイルのパスを設定します
model_config.tokenizer = f"{WEIGHTS_PATH}/tokenizer.model"
# VARIANTに"quant"が含まれているかどうかを設定します
model_config.quant = "quant" in VARIANT
# デフォルトのデータ型をモデル設定に基づいて設定します
torch.set_default_dtype(model_config.get_dtype())

# Gemmaモデルを読み込みます
gemma_model = GemmaForCausalLM(model_config)
# モデルの重みを指定したパスから読み込みます
gemma_model.load_weights(f"{WEIGHTS_PATH}/gemma-{VARIANT}.ckpt")
# モデルを指定したデバイスに移動し、評価モードに設定します
gemma_model = gemma_model.to(DEVICE).eval()

## Gemmaの使用

### 簡単なQA（質問応答）

`generate`メソッドを使用して、モデルを利用することができます。

In [ ]:
prompt = "Let me know how to use you."

# gemma_modelのgenerateメソッドを使用して、モデルの回答を生成します。
model_answer = gemma_model.generate(
    prompts=prompt,  # モデルに与えるプロンプト
    device=torch.device(DEVICE),  # 使用するデバイスを指定
    output_len=100,  # 出力する最大トークン数
    temperature=0.01,  # 出力の多様性を制御する温度
    top_p=0.1,  # 確率的サンプルのトップPフィルタリング
    top_k=1,  # トップKフィルタリングの設定
)

# モデルの回答を表示します
print(model_answer)

### インストラクションチューニングによるQA

インストラクションチューニングを使用して、プロンプトエンジニアリングを行うことができます。  
詳細なコードは、[スターターノートブック](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)の`GemmaFormatter`クラスで確認できます。

参考文献: https://ai.google.dev/gemma/docs/formatting

In [ ]:
system_prompt = "あなたは「20の質問」ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者の「はい」または「いいえ」の質問に応じます。そのキーワードは特定の人、場所、または物です。"

# プロンプトを作成します
prompt = "<start_of_turn>user\n"  # ユーザーのターンの開始を示す
prompt += f"{system_prompt}<end_of_turn>\n"  # システムプロンプトを追加し、ターンの終了を示す
prompt += "<start_of_turn>model\n"  # モデルのターンの開始を示す

# 作成したプロンプトを表示します
print(prompt)

In [ ]:
model_answer = gemma_model.generate(
    prompts=prompt,  # 上で作成したプロンプトをモデルに渡します
    device=torch.device(DEVICE),  # 使用するデバイスを指定
    output_len=100,  # 出力する最大トークン数
    temperature=0.01,  # 出力の多様性を制御する温度
    top_p=0.1,  # 確率的サンプルのトップPフィルタリング
    top_k=1,  # トップKフィルタリングの設定
)

# モデルの回答を表示します
print(model_answer)

---

これで、コンペティションのためにGemmaモデルを使用する方法がわかることを願っています。  
このノートブックがスターターノートブックを理解するのに役立つことを願っています。